## Project/Workflow Mockup
This notebook is intended to show some of the capabilities needed in a CMB workflow-management system. The main software components used in this mockup are:

* **Workflow**: An specification/template of the steps and data needed to accomplish one or more simulation objectives. A CMB workflow does not define a specific sequence of activities to perform, but instead enumerates a set of *tasks* that are required and the data *assets* that are used in the process.
* **Project**: A container for the data assets that are produced when a workflow is applied to a specific simulation objective.
* **Task**: An activity or set of activities required to produce or modify one or more simulation assets. For this mockup, tasks are presumed to be manual activities (i.e., there are no compuational pipelines).
* **Asset**: A data component required to complete the workflow. Asset contents are not well-defined; for now, an asset can be an smtk resource, a single file, or a folder/collection of files. Each asset in a workflow is assigned a unique "role" (string) which is used as its workflow identifier. Each task is specified with two list of assets, one for  prerequisite or "consuming" assets, and the other for assets that are created or modified by the task, aka "producing" assets.

The order that tasks are performed and assets produced is not explicitly prescribed by a workflow. Instead, the workflow software tracks which tasks *can* be performed based on the presence/absence of project assets. It is anticipated that typical engineering projects/workflow will be iterative, with tasks performed multiple times based on incremental results and feedback. The main purpose of this mockup is to demonstrate the task-asset dependency.

Note that no SMTK code is used; this is only a mockup.

## Preload workflow and instantiate project

This example uses a predefined, simplifed workflow to create an MCC analysis in 3 steps: (i) create an RGG model, (ii) create simulation attributes, and (iii) generate the analysis (combining modelbuilder export, PyARC processing and MCC simulation).

In [ ]:
import project    # NOT the smtk project module
import workflow

# Load the hard-coded workflow
mock_workflow = workflow.create('neams.1')
print('Created workflow: \"{}\"'.format(mock_workflow))

# Initialize a project with this workflow
mock_project = project.Project(mock_workflow)
print('Initialized project')

## List project tasks

Tasks are specified in the workflow object.

In [ ]:
from tabulate import tabulate  # for pretty printing

task_headers = ['Id', 'Title', 'Description']
task_table = [[t.id, t.title, t.description] for t in mock_project.tasks()]
print(tabulate(task_table, headers=task_headers))

## List project assets
Assets are specified in the workflow object.

The "Role" strings can be arbitrarily chosen, but must be unique within a workflow.

The "P/C" column indicates whether the asset is produced and/or consumed by this project. In this mockup, everything is produced somewhere in the workflow.

In [ ]:
asset_headers = ['Id', 'Role', 'P/C', 'Type', 'Description']
asset_table = [[a.id, mock_project.role(a), mock_project.pc(a), a.asset_type, a.description] \
    for a in mock_project.assets()]
print(tabulate(asset_table, headers=asset_headers))

## Print project/workflow summary

This lists the tasks with their assigned assets. Note that the same asset is typically produced in one task and consumed in another task(s).

In [ ]:
summary_headers = ['Task Id', 'Asset Id', 'Role', 'P/C', 'Type']
summary_table = None
for t in mock_project.tasks():
    if summary_table is None:
        summary_table = [[t.id, a.id, mock_project.role(a), t.pc(a), a.asset_type] for a in t.assets]
    else:
        summary_table.append([])
        summary_table += [[t.id, a.id, mock_project.role(a), t.pc(a), a.asset_type] for a in t.assets]
print(tabulate(summary_table, headers=summary_headers))

## Print intial task status

This lists the 3 project tasks and indicates if each task is ready to be performed, based on whether the consumed assets have been assigned to the project or not. Since no assets have been created and assigned to the project yet, only the first task is ready, because it requires (consumes) no assets.

In [ ]:
status_headers = ['Task Id', 'Title', 'Ready?']
status_table = [[t.id, t.title, t.is_ready()] for t in mock_project.tasks()]
print(tabulate(status_table, headers=status_headers))

## Make a mock assignment of the reactor_geometry asset
In this mockup, assets are "assigned" to the project manually, to imply that the underlying data has been created. In practice, projects might handle this more gracefully. The main point here is that the second task ("Specify MCC simulation") is now ready because a reactor_geometry asset has been "assigned" to the project. Another way of stating this might be that the reactor_geometry asset now "has data".

In [ ]:
mock_project.assign_asset('reactor_geometry', 'some data tbd')

status_headers = ['Task Id', 'Title', 'Ready?']
status_table = [[t.id, t.title, t.is_ready()] for t in mock_project.tasks()]
print(tabulate(status_table, headers=status_headers))

## Change assignments
You can make additional assignments to the other project assets, or unassign the reactor_geometry (by assiging None). Recomputing the status table should reflect those changes. Note that no checking is done to mark assets as "possibly invalid" when upstream assets are changed.

You should be able to continue indefinitely, but if you are using a binder page, there is a timeout. Also, if the page gets stuck or you hit an Exception, you can restart the kernel and proceed from the first cell. (Use the menu "Kernel" --> "Restart & Clear Output")